In [1]:
# Import pyNBS modules
from pyNBS import data_import_tools as dit
from pyNBS import network_propagation as prop
from pyNBS import pyNBS_core as core
from pyNBS import pyNBS_single
from pyNBS import consensus_clustering as cc
from pyNBS import pyNBS_plotting as plot

# Import other needed packages
import os
import time
import pandas as pd
import numpy as np
from IPython.display import Image

# Load Data
First, we must load the somatic mutation and network data for running pyNBS. We will also set an output directory location to save our results.

### Load binary somatic mutation data
The binary somatic mutation data file can be represented in two file formats:  
The default format for the binary somatic mutation data file is the ```list``` format. This file format is a 2-column csv or tsv list where the 1st column is a sample/patient and the 2nd column is a gene mutated in the sample/patient. There are no headers in this file format. Loading data with the list format is typically faster than loading data from the matrix format.The following text is the list representation of the matrix above.
```
TCGA-04-1638	A2M
TCGA-23-1029	A1CF
TCGA-23-2647	A2BP1
TCGA-24-1847	A2M
TCGA-42-2589	A1CF
```

The ```matrix``` binary somatic mutation data format is the format that data for this example is currently represented. This file format is a binary csv or tsv matrix with rows represent samples/patients and columns represent genes.  The following table is a small excerpt of a matrix somatic mutation data file:  

||A1CF|A2BP1|A2M|
|-|-|-|-|
|TCGA-04-1638|0|0|1|
|TCGA-23-1029|1|0|0|
|TCGA-23-2647|0|1|0|
|TCGA-24-1847|0|0|1|
|TCGA-42-2589|1|0|0|

__Note:__ The default file type is defined as ```'list'```, but if the user would like to specify the 'matrix' type, the user needs to simply pass the string ```'matrix'``` to the ```filetype``` optional parameter (as below). The delimiter for the file is passed similarly to the optional parameter ```delimiter```

For more examples and definitions in the somatic mutation data file format, please see our Github Wiki page:  
https://github.com/huangger/pyNBS/wiki/Somatic-Mutation-Data-File-Format

In [2]:
sm_data_filepath = './Example_Data/Mutation_Files/BLCA_sm_data.txt'
sm_mat = dit.load_binary_mutation_data(sm_data_filepath, filetype='list', delimiter='\t')

Binary Mutation Matrix Loaded: ./Example_Data/Mutation_Files/BLCA_sm_data.txt


### Load molecular network
The network file is a 2-column text file representing an unweighted network. Each row represents a single edge in the molecular network.    
  
Notes about the network file:  
 - The default column delimiter is a tab character '\t' but a different delimiter can be defined by the user here or in the parameter file with the "net_filedelim" parameter.
 - The network must not contain duplicate edges (e.g. TP53\tMDM2 is equivalent to MDM2\tTP53)
 - The network must not contain self-edges (e.g. TP53\tTP53)
 - Only the first two columns of a network file are read as edges for the network, all other columns will be ignored.
 - The load_network function also includes options to read in edge- or label-shuffled versions of the network, but by default, these options are turned off.
 
An excerpt of the first five rows of the PID network file is given below:  
```
A1BG	A2M
A1BG	AKT1
A1BG	GRB2
A1BG	PIK3CA
A1BG	PIK3R1
```

For more examples and definitions in the network file format, please see our Github Wiki page:  
https://github.com/huangger/pyNBS/wiki/Molecular-Network-File-Format

In [3]:
# The only required parameter for this function is the network file path

network_filepath = './Example_Data/Network_Files/CancerSubnetwork.txt'
network = dit.load_network_file(network_filepath)
network.nodes

Network File Loaded: ./Example_Data/Network_Files/CancerSubnetwork.txt


NodeView(('HSPA2', 'RPN1', 'GK2', 'HSPA6', 'PPP3R1', 'DLG1', 'YWHAH', 'HIST1H4I', 'HSPA8', 'PCSK6', 'PCSK7', 'PRPF8', 'TPI1', 'AKT3', 'IKBKG', 'SEC24D', 'DHX15', 'RFWD2', 'TYK2', 'PFKP', 'PPP3R2', 'MAP3K7', 'ACTG1', 'RARA', 'EP300', 'CACYBP', 'ACTG2', 'CDC73', 'PABPC3', 'CACNA1A', 'CACNA1B', 'CACNA1C', 'CACNA1E', 'CSNK2A2', 'HSP90AB1', 'PLK1', 'PERP', 'CAD', 'LIG1', 'HDAC2', 'NUP214', 'JAK2', 'LYN', 'CUL1', 'LIPA', 'IGF1R', 'DDX3X', 'GPI', 'CDKN2D', 'NUP98', 'CAMK2D', 'IDH2', 'IDH1', 'HK1', 'NUP93', 'CREB1', 'CD24', 'FXR1', 'TNFRSF21', 'STK3', 'STK4', 'TPR', 'EGFR', 'RFC2', 'GK', 'FGFR3', 'SHMT1', 'GMPS', 'MAPK14', 'ATIC', 'FN1', 'RPS6', 'PPP2CA', 'ACSL3', 'EIF4G1', 'FZR1', 'RBBP7', 'AKT2', 'GSK3B', 'TERF2', 'SYK', 'DDX6', 'XRN1', 'RAD23B', 'UBC', 'PGAM2', 'NTHL1', 'PGK1', 'YBX1', 'ELOC', 'PCNA', 'CTNND1', 'SEC31A', 'ALDH3A2', 'GSK3A', 'EIF4E1B', 'RPL5', 'GAPDH', 'HSP90B1', 'MYLK', 'RRAS', 'ACAA1', 'AKR1A1', 'FEN1', 'CLTC', 'TNPO1', 'STIP1', 'VIM', 'ACADM', 'HSPA1A', 'XRCC6', 'XPO1', '

### Setting result output options
The following code is completely optional for the user. Allows users to pre-define a directory to save intermediate and final results to and establishes a file name prefix for those files in the output directory folder. Also creates the output directory if it does not already exist. The result of this cell will be a dictionary that can be passed optionally to functions to save results.

**Note:** The key assumption here is that if the user passes **save_args to the function that contains a valid file path to a directory in ```outdir```, the result of that particular call of the function will be saved to the given ```outdir```

In [4]:
# Optional: Setting the output directory for files to be saved in
outdir = './Results/via_notebook/CancerSubnetwork_BLCA/'

# Optional: Creating above output directory if it doesn't already exist
if not os.path.exists(outdir):
    os.makedirs(outdir)
    
# Optional: Setting a filename prefix for all files saved to outdir
job_name = 'CancerSubnetwork_BLCA'

# Constructs dictionary to be passed as "save_args" to functions if output to be saved
save_args = {'outdir': outdir, 'job_name': job_name}

# Construct regularization graph for use in network-regularized NMF

In this step, we will construct the graph used in the network-regularized non-negative matrix factorization (netNMF) step of pyNBS. This network is a K-nearest neighbor (KNN) network constructed from the network influence matrix (Vandin et al 2011*) of the molecular network being used to stratify tumor samples. The graph laplacian of this KNN network (knnGlap) is used as the regularizer in the following netNMF steps. This step uses the ```network_inf_KNN_glap``` function in the pyNBS_core module. 

For additional notes on the graph laplacian construction method, please visit our GitHub wiki for this function:
https://github.com/huangger/pyNBS/wiki/pyNBS.pyNBS_core.network_inf_KNN_glap

---
**Note: ** This step is technically optional. No regularization network laplacian has to be constructed if the user would like to run the NMF step without a network regularizer. The user simply has to pass ```None``` into the optional parameter ```regNet_glap``` or remove the optional parameter in the ```pyNBS_single()``` function call below. This will cause pyNBS to run a non-network regularized NMF procedure. However, given the implementation of the multiplicative update steps, the results may not be exactly the same as some other NMF implementations (e.g. from scikit-learn).

In [5]:
# Constructing knnGlap
knnGlap = core.network_inf_KNN_glap(network)

##########################################################################################################
# The resulting matrix can be very large, so we choose not to save the intermediate result here
# To run this function and save the KNN graph laplaican to the output directory 'outdir' given above:
# Uncomment and run the following line instead:
# knnGlap = core.network_inf_KNN_glap(network, **save_args)
##########################################################################################################

Graph influence matrix calculated: 1.2630746364593506 seconds
Graph laplacian of KNN network from influence matrix calculated: 2.501058340072632 seconds


In [6]:
knnGlap

,HSPA2,PCSK6,EIF4E1B,CD24,PGAM4,PGK2,CARS,GK2,ACSS1,MLH3,...,WNT8B,MAP2K6,FZD7,WNT10B,GDF5,WNT2B,CDX2,IRAK2,WNT7B,WNT16
HSPA2,10,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,0,0,0,0,0,0,0,0,0,0
PCSK6,-1,24,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
EIF4E1B,-1,0,38,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
CD24,-1,0,0,30,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
PGAM4,-1,0,0,0,48,-1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WNT2B,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,10,0,0,0,0
CDX2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,10,0,0,0
IRAK2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,10,0,0
WNT7B,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,10,0


# Construct network propagation kernel matrix
Due to the multiple subsampling and propagation steps used in pyNBS, we have found that the algorithm can be significantly sped up for large numbers of subsampling and propagation iterations if a gene-by-gene matrix describing the influence of each gene on every other gene in the network by the random-walk propagation operation is pre-computed. We refer to this matrix as the "network propagation kernel". Here we compute this propagation kernel by propagating the all genes in the molecular network independently of one another. The propagation profile of each tumor is then simply the column sum vector of the resulting network propagation kernel selected for only the rows of genes marked as mutated in each tumor, rather than having to perform the full network propagation step again after each subsampling of the data. 

For additional notes on the propagation methods used, please visit our GitHub wiki for this function:
https://github.com/huangger/pyNBS/wiki/pyNBS.network_propagation.network_propagation


### Calibrating the network propagation coefficient
The current network propagation coefficient ($\alpha$) is currently set to 0.7 and must range between 0 and 1. This parameter can be tuned and changing it may have a result on the final propagation results. Previous results from [Hofree et al 2013](https://www.nature.com/articles/nmeth.2651) suggest that values between 0.5 and 0.8 produce relatively robust results, but we suspect that the optimal value may be dependent on certain network properties such as edge density.

In [7]:
# Set or change network propagation coefficient if desired
alpha = 0.7

In [8]:
# Construct identity matrix of network
network_nodes = network.nodes()
network_I = pd.DataFrame(np.identity(len(network_nodes)), index=network_nodes, columns=network_nodes)

In [9]:
# Construct network propagation kernel
kernel = prop.network_propagation(network, network_I, alpha=alpha, symmetric_norm=False)

##########################################################################################################
# The resulting matrix can be very large, so we choose not to save the intermediate result here
# To run this function and save the propagation kernel to the output directory 'outdir' given above,
# Uncomment and run the following two lines instead of the above line:
# save_args['iteration_label']='kernel'
# kernel = prop.network_propagation(network, network_I, alpha=alpha, symmetric_norm=True, **save_args)
##########################################################################################################

Performing network propagation with alpha: 0.7
Network Propagation Complete: 1.990267038345337 seconds


In [10]:
kernel

,HSPA2,RPN1,GK2,HSPA6,PPP3R1,DLG1,YWHAH,HIST1H4I,HSPA8,PCSK6,...,JMJD7-PLA2G4B,MALAT1,REG3G,NUTM2A,TRB,IGL,HES3,CCL15,CCL4L2,SCUBE1
HSPA2,0.301161,0.000956,0.000756,0.001607,0.000843,0.000995,0.001226,0.001015,0.001913,0.000658,...,0.000010,0.000001,8.587660e-07,1.228380e-06,9.786070e-07,8.628454e-07,0.000002,1.167224e-06,1.425421e-06,0.000004
RPN1,0.002033,0.300840,0.000080,0.002078,0.000182,0.000354,0.001989,0.000374,0.002458,0.000031,...,0.000031,0.000001,8.347618e-07,1.506721e-06,1.171929e-06,1.253445e-06,0.000002,1.103068e-06,1.402252e-06,0.000002
GK2,0.004539,0.000225,0.301491,0.004585,0.000243,0.000275,0.000380,0.000287,0.004798,0.000071,...,0.000014,0.000001,1.854933e-06,1.054814e-06,1.049891e-06,1.097301e-06,0.000004,2.528822e-06,2.167794e-06,0.000002
HSPA6,0.001356,0.000825,0.000645,0.301138,0.000712,0.000862,0.001064,0.000894,0.001676,0.000029,...,0.000013,0.000004,1.249675e-06,1.129278e-06,1.257611e-06,7.627595e-07,0.000002,3.436474e-06,3.418481e-06,0.000003
PPP3R1,0.001901,0.000193,0.000091,0.001902,0.301018,0.000488,0.001999,0.000301,0.002241,0.000043,...,0.000012,0.000001,1.101603e-06,1.538312e-06,1.154950e-06,7.076435e-07,0.000002,1.579535e-06,1.698535e-06,0.000002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
IGL,0.000290,0.000198,0.000061,0.000304,0.000105,0.000466,0.000177,0.000092,0.000406,0.000020,...,0.000006,0.000001,6.062103e-07,6.784775e-07,5.928115e-07,3.420842e-01,0.000002,8.321551e-07,8.988069e-07,0.000001
HES3,0.000189,0.000085,0.000066,0.000215,0.000096,0.000214,0.000227,0.000173,0.000371,0.000043,...,0.000007,0.000001,1.084281e-06,6.428052e-07,7.739368e-07,7.895690e-07,0.306965,1.350374e-06,1.528725e-06,0.000002
CCL15,0.000131,0.000058,0.000047,0.000456,0.000078,0.000144,0.000159,0.000098,0.000271,0.000021,...,0.000006,0.000001,3.651458e-06,4.230124e-07,2.179614e-06,2.773850e-07,0.000001,3.007345e-01,1.846477e-04,0.000001
CCL4L2,0.000160,0.000074,0.000040,0.000454,0.000084,0.000163,0.000198,0.000123,0.000315,0.000024,...,0.000020,0.000005,1.208990e-05,5.961497e-07,1.772411e-06,2.996023e-07,0.000002,1.846477e-04,3.013736e-01,0.000002


# Subsampling, propagation, and netNMF
After the pre-computation of the regularization graph laplacian and the network propagation kernel, we perform the following core steps of the NBS algorithm multiple times (default=100x) to produce multiple patient clusterings that will be used in the later consensus clustering step. Each patient clustering is performed with the following steps:  
  
 1. **Subsample binary somatic mutation data.** (See the documentation for the [```subsample_sm_mat```](https://github.com/huangger/pyNBS/wiki/pyNBS.pyNBS_core.subsample_sm_mat) function for more details.)
 2. **Propagate binary somatic mutation data over network.** (See the documentation for the [```network_propagation```](https://github.com/huangger/pyNBS/wiki/pyNBS.network_propagation.network_propagation) or [```network_kernel_propagation```](https://github.com/huangger/pyNBS/wiki/pyNBS.network_propagation.network_propagation) function for more details.)
 3. **Quantile normalize the network-smoothed mutation data.** (See the documentation for the [```qnorm```](https://github.com/huangger/pyNBS/wiki/pyNBS.pyNBS_core.qnorm) function for more details.) 
 4. **Use netNMF to decompose network data into k clusters.** (See the documentation for the [```mixed_netNMF```](https://github.com/huangger/pyNBS/wiki/pyNBS.pyNBS_core.mixed_netNMF) function for more details.)
  
These functions for each step here are wrapped by the [```NBS_single```](https://github.com/huangger/pyNBS/wiki/pyNBS.pyNBS_single.NBS_single) function, which calls each step above in sequence to perform a single iteration of the pyNBS algorithm.

### Number of pyNBS clusters
The default number of clusters constructed by pyNBS is k=3. We change that definition explicitly below or in the parameters for [```NBS_single```](https://github.com/huangger/pyNBS/wiki/pyNBS.pyNBS_single.NBS_single), and in this example we choose 4 clusters. Other parameters such as the subsampling parameters and the propagation coefficient (when no kernel is pre-computed) can also be changed using \*\*kwargs. \*\*kwargs can also will hold the values of \*\*save_args as seen in previous functions if the user would like to save the resulting dimension reduced patient profiles. All documentation of \*\*kwargs definitions are given in the Github wiki page for [```NBS_single```](https://github.com/huangger/pyNBS/wiki/pyNBS.pyNBS_single.NBS_single)

In [11]:
clusters = 4

### Number of pyNBS iterations
The consensus clustering step of the pyNBS algorithm will improve if the data is subsampled, and re-clustered multiple times. The default number of times we perform the aforementioned operation (```niter```) is 100 times. The number can be reduced for faster run-time, but may produce less robust results. Increasing ```niter``` will increase overall runtime, but should produce more robust cluster assignments during consensus clustering.

In [12]:
# Set the number of times to perform pyNBS core steps
niter = 5

In [13]:
# Optional: Saving the intermediate propagation step (from subsampled data) to file
# save_args['save_prop'] = True

# Run pyNBS 'niter' number of times
Hlist = []
for i in range(niter):
    netNMF_time = time.time()
    # Run pyNBS core steps and save resulting H matrix to Hlist
    Hlist.append(pyNBS_single.NBS_single(sm_mat, knnGlap, propNet=network, propNet_kernel=kernel, k=clusters))
    
    ##########################################################################################################
    # Optional: If the user is saving intermediate outputs (propagation results or H matrices), 
    # a different 'iteration_label' should be used for each call of pyNBS_single().
    # Otherwise, the user will overwrite each H matrix at each call of pyNBS_single()
    # Uncomment and run the two lines below to save intermediate steps instead of the previous line
    # save_args['iteration_label']=str(i+1)
    # Hlist.append(pyNBS_single.NBS_single(sm_mat, propNet=network, propNet_kernel=kernel, regNet_glap=knnGlap, 
    #                                      k=clusters, **save_args))
    ##########################################################################################################
    
    # Report run time of each pyNBS iteration
    t = time.time()-netNMF_time
    print ('NBS iteration:', i+1, 'complete:', t, 'seconds')

/home/aswin/.local/lib/python3.9/site-packages/pyNBS/pyNBS_core.py:151: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  W_init = np.linalg.lstsq(H.T, data.T)[0].T


NBS iteration: 1 complete: 27.006142139434814 seconds
NBS iteration: 2 complete: 29.900998830795288 seconds
NBS iteration: 3 complete: 30.336366415023804 seconds
NBS iteration: 4 complete: 28.95118236541748 seconds
NBS iteration: 5 complete: 29.88498330116272 seconds


# Consensus Clustering
In order to produce robust patient clusters, the sub-sampling and re-clustering steps as done above are needed. After the patient data is subsampled multiple times (default ```niter```=100), we perform the [```consensus_hclust_hard```](https://github.com/huangger/pyNBS/wiki/pyNBS.consensus_clustering.consensus_hclust_hard) function in the conensus_clustering module. It accepts a list of pandas dataframes as generated in the previous step. If the H matrices were generated separately and saved to a directory, the user will need to manually import those H matrices into a python list first before passing the list to the function below. 

For more information on how the consensus clustering is performed, please see our wiki page on this function:
https://github.com/huangger/pyNBS/wiki/pyNBS.consensus_clustering.consensus_hclust_hard

In [14]:
NBS_cc_table, NBS_cc_linkage, NBS_cluster_assign = cc.consensus_hclust_hard(Hlist, k=clusters, **save_args)

Constructing Hlist: 5 cluster matrices,  393 samples


KeyError: 0

# Co-Clustering Map
To visualize the clusters formed by the pyNBS algorithm, we can plot a similarity map using the objects created in the previous step. This step uses the [`cluster_color_assign`](https://github.com/huangger/pyNBS/wiki/pyNBS.pyNBS_plotting.cluster_color_assign) and [`plot_cc_map()`](https://github.com/huangger/pyNBS/wiki/pyNBS.pyNBS_plotting.plot_cc_map) functions in the [`pyNBS_plotting`](https://github.com/huangger/pyNBS/wiki/pyNBS.pyNBS_plotting) module.

In [ ]:
# Assign colors to clusters from pyNBS
pyNBS_BLCA_clust_cmap = plot.cluster_color_assign(NBS_cluster_assign, name='pyNBS BLCA Cluster Assignments')

# Plot and save co-cluster map figure
plot.plot_cc_map(NBS_cc_table, NBS_cc_linkage, col_color_map=pyNBS_BLCA_clust_cmap, **save_args)

In [ ]:
Image(filename = save_args['outdir']+save_args['job_name']+'_cc_map.png', width=600, height=600)

# Survival analysis
To determine if the patient clusters are prognostically relevant, we perform a standard survival analysis using a multi-class logrank test to evaluate the significance of survival separation between patient clusters. This data is plotted using a Kaplan-Meier plot using the [`cluster_KMplot()`](https://github.com/huangger/pyNBS/wiki/pyNBS.pyNBS_plotting.cluster_KMplot) in the [`pyNBS_plotting`](https://github.com/huangger/pyNBS/wiki/pyNBS.pyNBS_plotting) module. 


In order to plot the survival differences between clusters, we will need to load survival data for each patient. This data was extracted from TCGA clinical data. The survival data is given in a 5-column delimited table with the specific headings described below (the columns must be in the same order as shown below). The following is an example of a few lines of the a survival table:  

||vital_status|days_to_death|days_to_last_followup|overall_survival|
|-|-|-|-|-|
|TCGA-2E-A9G8|0|0|1065|1065|
|TCGA-A5-A0GI|0|0|1750|1750|
|TCGA-A5-A0GM|0|0|1448|1448|
|TCGA-A5-A1OK|0|0|244|244|
|TCGA-A5-AB3J|0|0|251|251|

Additional details on the survival data file format is also describe on our Github wiki at: 
https://github.com/huangger/pyNBS/wiki/Patient-Survival-Data-File-Format

Note: The default setting for pyNBS is that no survival curves are drawn because the survival data is not a required parameter. The path to valid survival data must be explicitly defined.

In [ ]:
# Load survival Data
surv_data = './Example_Data/Clinical_Files/BLCA.clin.merged.surv.txt'

# Plot KM Plot for patient clusters
plot.cluster_KMplot(NBS_cluster_assign, surv_data, delimiter=',', **save_args)
Image(filename = save_args['outdir']+save_args['job_name']+'_KM_plot.png', width=600, height=600)